In [55]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import itertools
import torch.nn.functional as F
import numpy as np
import pickle

if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'
print(device)

cuda


### Loading Transformers

In [56]:
tokenizer = AutoTokenizer.from_pretrained("MohammedDhiyaEddine/job-skill-sentence-transformer-tsdae")
model = AutoModel.from_pretrained("MohammedDhiyaEddine/job-skill-sentence-transformer-tsdae").to(device)
# tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/jobbert-self-supervised/JOBBERT_FINETUNED_tokenizer")
# model = AutoModel.from_pretrained("/kaggle/input/jobbert-self-supervised/JOBBERT_FINETUNED").to(device)

In [57]:
# Open the pickle file for reading
with open('/kaggle/input/embeddings/jobEmbeddingArya.pickle', 'rb') as f:
#     Use pickle to load the variable from the file
    job_embeddings = pickle.load(f)

# Print the loaded variable
print(job_embeddings.shape)

torch.Size([828, 768])


In [58]:
user_df=pd.read_csv("/kaggle/input/userresume/UserResume.csv").dropna().drop(["Resume"],axis=1)
user_df.head()
user_df["Combined"] = user_df["Category"].astype(str) + user_df["Cleaned_Resume"]
user_df=user_df.drop(["Category","Cleaned_Resume"],axis=1)
user_df.head()

,Combined
0,Data ScienceSkills Programming Languages Pytho...
1,Data ScienceEducation Details May 2013 to May ...
2,Data ScienceAreas of Interest Deep Learning Co...
3,Data ScienceSkills R Python SAP HANA Tableau S...
4,Data ScienceEducation Details MCA YMCAUST Fari...


In [59]:
def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
def getUserEmbedding(userResume):
    embeddings=torch.Tensor([]).to(device)#(np.zeros((n,df.shape[1],max_len,768)))
    sentences =userResume
    encoded_sentences = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    encoded_sentences.to(device)
    with torch.no_grad():
        temp=model(**encoded_sentences)
    sentence_embeddings = mean_pooling(temp, encoded_sentences['attention_mask'])
    embeddings=sentence_embeddings
    return embeddings
getUserEmbedding(user_df.iloc[0].Combined).shape

torch.Size([1, 768])

In [ ]:
getUserEmbedding("I am familiar with python")

In [61]:
# df=pd.read_csv("/kaggle/input/scrapeddata/cleaned_jobs.csv")
df=pd.read_excel("/kaggle/input/jobs-xlsx/Jobs.xlsx").reset_index()
df=df.rename(columns={'index':'Index','Job Description':'Description'})
df=df.set_index('Index')
df.head()

,Position,Company,Location,Description
Index,,,,
0,Kinaxis Application Developer,Accenture in India,"Bengaluru, Karnataka, India",About the job\nAbout Accenture: Accenture is a...
1,ForgeRock Identity Management Security Deliver...,Accenture in India,"Bengaluru, Karnataka, India",About the job
2,Clinical - SDTM Programming Senior Analyst,Accenture in India,"Bengaluru, Karnataka, India",About the job\nSkill required: Clinical - SDTM...
3,Ping Identity Solutions Security Architect,Accenture in India,"Bengaluru, Karnataka, India",About the job\nAbout Accenture: Accenture is a...
4,PL/ SQL Developer,NR Consulting,"Bengaluru, Karnataka, India",About the job\n Detailed JD-\n\nDevelop proced...


In [62]:
import re
from bs4 import BeautifulSoup
def cleanResume(resumeText):
    soup=BeautifulSoup(resumeText,"html.parser")
    resumeText=soup.get_text()
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText
    
df['Cleaned_Description'] = df.Description.apply(lambda x: cleanResume(x))
df

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


,Position,Company,Location,Description,Cleaned_Description
Index,,,,,
0,Kinaxis Application Developer,Accenture in India,"Bengaluru, Karnataka, India",About the job\nAbout Accenture: Accenture is a...,About the job About A enture A enture is a glo...
1,ForgeRock Identity Management Security Deliver...,Accenture in India,"Bengaluru, Karnataka, India",About the job,About the job
2,Clinical - SDTM Programming Senior Analyst,Accenture in India,"Bengaluru, Karnataka, India",About the job\nSkill required: Clinical - SDTM...,About the job Skill required Clinical SDTM Pro...
3,Ping Identity Solutions Security Architect,Accenture in India,"Bengaluru, Karnataka, India",About the job\nAbout Accenture: Accenture is a...,About the job About A enture A enture is a glo...
4,PL/ SQL Developer,NR Consulting,"Bengaluru, Karnataka, India",About the job\n Detailed JD-\n\nDevelop proced...,About the job Detailed JD Develop procedure an...
...,...,...,...,...,...
969,Data Engineer - Azure Data Factory/DataLake,Zelus India,"Bengaluru, Karnataka, India",About the job\n\nDeveloper is responsible for ...,About the job Developer is responsible for cre...
970,Python Developer,CloudPanel Technologies,"Bengaluru North, Karnataka, India",About the job\nThis job is sourced from a job ...,About the job This job is sourced from a job b...
971,C++ Developer,Onnet Systems India Pvt Ltd,"Bengaluru, Karnataka, India",About the job\nJob Description\n\nThe ideal ca...,About the job Job Description The ideal candid...


In [63]:
import math
def getCosineMatrix(userResume):
    user_embedding=getUserEmbedding(userResume)
#     print(user_embedding.shape)
#     print(job_embeddings.shape)
    # Create two example tensors
    tensor1 = user_embedding
    tensor2 = job_embeddings

    # Generate all permutations of the two tensors
    permutations = itertools.product(tensor1, tensor2)

    # Convert tuples to rows in a DataFrame
    Embedding_df = pd.DataFrame(permutations, columns=['tensor1', 'tensor2'])

    # Display the resulting DataFrame
#     print(Embedding_df.shape)
    Embedding_df['Index']=range(tensor2.shape[0])
    Embedding_df.set_index('Index')
    Embedding_df['cosine_similarity']=np.zeros((Embedding_df.shape[0]))
#     print(Embedding_df.shape)

    for i in range(math.ceil(Embedding_df.shape[0])):
        tensor1 = Embedding_df.iloc[i]['tensor1']
        tensor2 = Embedding_df.iloc[i]['tensor2']
    # compute the cosine similarity between the two tensors
        cosine_similarity_value = F.cosine_similarity(tensor1.unsqueeze(0), tensor2.unsqueeze(0)).item()
        Embedding_df['cosine_similarity'].iloc[i]=cosine_similarity_value
    Embedding_df = Embedding_df.sort_values(by='cosine_similarity', ascending=False)
    
    merged_df = pd.merge(Embedding_df, df, on='Index').set_index('Index')
#     merged_df
    
    return Embedding_df,merged_df

CosineMatrix,merged_df=getCosineMatrix("I am familiar with python")
CosineMatrix.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,tensor1,tensor2,Index,cosine_similarity
323,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.6617, device='cuda:0'), tensor(-0.0...",323,0.782478
614,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.8616, device='cuda:0'), tensor(0.30...",614,0.778540
769,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.7978, device='cuda:0'), tensor(0.32...",769,0.776148
141,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.7639, device='cuda:0'), tensor(0.15...",141,0.772601
639,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.4676, device='cuda:0'), tensor(0.28...",639,0.759796


In [64]:
merged_df.head()

,tensor1,tensor2,cosine_similarity,Position,Company,Location,Description,Cleaned_Description
Index,,,,,,,,
323,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.6617, device='cuda:0'), tensor(-0.0...",0.782478,"Python Developer | Bangalore, India",Eastvantage,"Bengaluru, Karnataka, India","About the job\nLocation:\n\nBangalore, India\n...",About the job Location Bangalore India Contrac...
614,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.8616, device='cuda:0'), tensor(0.30...",0.778540,Data Analyst,Glean,"Bengaluru, Karnataka, India",About the job\nAbout Glean\n\nWe're on a missi...,About the job About Glean We re on a mission t...
769,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.7978, device='cuda:0'), tensor(0.32...",0.776148,"Java Developer - Java, Microservice, Kafka, Sp...",XCUTIVES Inc.,"Bengaluru, Karnataka, India",About the job\nPosition : Senior Software Engi...,About the job Position Senior Software Enginee...
141,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.7639, device='cuda:0'), tensor(0.15...",0.772601,Data Engineer(ADF.T-SQL),Diverse Lynx,"Bengaluru, Karnataka, India",About the job\nCreate and maintain optimal dat...,About the job Create and maintain optimal data...
639,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.4676, device='cuda:0'), tensor(0.28...",0.759796,java developer,Diverse Lynx,"Bengaluru, Karnataka, India",About the job\nLooking for Java developer\n\n3...,About the job Looking for Java developer 3 Yea...


In [65]:
merged_df.drop(["Company","Location"],axis=1,inplace=True)
merged_df

,tensor1,tensor2,cosine_similarity,Position,Description,Cleaned_Description
Index,,,,,,
323,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.6617, device='cuda:0'), tensor(-0.0...",0.782478,"Python Developer | Bangalore, India","About the job\nLocation:\n\nBangalore, India\n...",About the job Location Bangalore India Contrac...
614,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.8616, device='cuda:0'), tensor(0.30...",0.778540,Data Analyst,About the job\nAbout Glean\n\nWe're on a missi...,About the job About Glean We re on a mission t...
769,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.7978, device='cuda:0'), tensor(0.32...",0.776148,"Java Developer - Java, Microservice, Kafka, Sp...",About the job\nPosition : Senior Software Engi...,About the job Position Senior Software Enginee...
141,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.7639, device='cuda:0'), tensor(0.15...",0.772601,Data Engineer(ADF.T-SQL),About the job\nCreate and maintain optimal dat...,About the job Create and maintain optimal data...
639,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.4676, device='cuda:0'), tensor(0.28...",0.759796,java developer,About the job\nLooking for Java developer\n\n3...,About the job Looking for Java developer 3 Yea...
...,...,...,...,...,...,...
453,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(0.1776, device='cuda:0'), tensor(0.378...",0.310196,DB developer (Data Base Developers),About the job,About the job
215,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(-0.3414, device='cuda:0'), tensor(0.44...",0.307775,Interesting Job Opportunity: Automation Test E...,About the job\nThis job is sourced from a job ...,About the job This job is sourced from a job b...
758,"[tensor(-0.1596, device='cuda:0'), tensor(-0.1...","[tensor(0.1364, device='cuda:0'), tensor(-0.15...",0.301227,Devops Implementation Engg,About the job\nKey Responsibilities: Infrastru...,About the job Key Responsibilities Infrastruct...


In [66]:
grouped_df = merged_df.groupby('Position').mean().sort_values(by='cosine_similarity', ascending=False)
grouped_df

,cosine_similarity
Position,
"Java Developer - Java, Microservice, Kafka, Spring boot",0.776148
Data Engineer(ADF.T-SQL),0.772601
Software Dev Principal Engineer - Python and Networking - 10+ yrs,0.753128
Data Analytics Engineer - Python/SQL,0.749961
Java Engineer - New - 52 <Test Job Visible to LinkedIn Employees Only>,0.742830
...,...
Data Analytics Senior Analyst,0.310880
Interesting Job Opportunity: Automation Test Engineer - Java/Selenium,0.307775
Devops Implementation Engg,0.301227


### MOnjoy ka jobbbert

In [67]:
tokenizer = AutoTokenizer.from_pretrained("jjzha/jobbert-base-cased")
model = AutoModel.from_pretrained("jjzha/jobbert-base-cased").to(device)

Some weights of the model checkpoint at jjzha/jobbert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at jjzha/jobbert-base-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.